### Definition

In [ ]:
! pip install openpyxl

In [1]:
import pandas as pd
# import numpy as np
# from datetime import date
# from dateutil.relativedelta import relativedelta

import warnings

# 忽略所有警告
warnings.filterwarnings('ignore')



In [2]:
# import data
df_shareholder_1 = pd.read_excel('十大股东文件2.xlsx')

In [3]:
df_shareholder_1 = df_shareholder_1[df_shareholder_1['持股排名']==1]

In [4]:
df1_1 = pd.read_excel('证券2.xlsx', engine='openpyxl')

In [5]:
df1_stock_1 = pd.merge(df1_1, df_shareholder_1, on=['证券代码','统计截止日期'],how='inner')


In [6]:
df1_stock_1.to_excel('证券2_final.xlsx', index=False)

In [ ]:
# import data
df1_1 = pd.read_excel('证券2.xlsx', engine='openpyxl')
df1_2 = pd.read_excel('证券1.xlsx', engine='openpyxl')
df1_3 = pd.concat([df1_1, df1_2], ignore_index=True)
df1 = df1_3[df1_3['民营指数'] >= 1]


In [ ]:
df2 = pd.read_excel('TRD_Mnth.xlsx', engine='openpyxl')
#df3 = pd.read_excel('TRD_Mnth.xlsx', engine='openpyxl')

In [ ]:
df3 = df2.copy()

In [ ]:
df2.info()

In [ ]:
df1.dtypes

In [ ]:
df1['统计日期'] = pd.to_datetime(df1['统计截止日期'])
df2['统计日期'] = pd.to_datetime(df2['交易月份'])
df3['统计日期'] = pd.to_datetime(df3['交易月份'])

### df1_sorted

In [ ]:
df1_sorted = df1.sort_values(by=['统计日期', '其他持股比例'], ascending=[True, False])
df1_sorted

In [ ]:
# grouped_returns = pd.DataFrame()

# # 9 三季报
# # 3 一季报
# # 6 半年报
# periods = {
#     9: (1, 4),
#     3: (5, 8),
#     6: (9, 12)
# }


In [ ]:
report_months = {
    'third': 9,
    'first': 3,
    'half': 6
}
    

In [ ]:
df1_sorted

In [ ]:
# add a new column in the df1
df1["分组"] = ""



In [ ]:
df1

In [ ]:
df1_copy = df1.copy()
len(df1_copy.index) 
df1_copy


### Dataset df1_group
对公司进行分组 按照季度进行调仓

In [ ]:

df1_group = pd.DataFrame()

df1_year_keymonth_size = 0

# 对公司进行分组 按照季度进行调仓
for year in df1['统计日期'].dt.year.unique():
    print('****** loop *******')
    print(year)
    
    
    for key, key_month in report_months.items():
        print('*****loop 季度*****')
        print(key)
        print(key_month)
         
        df1_year_keymonth = df1.loc[(df1['统计日期'].dt.year == year ) & (df1['统计日期'].dt.month == key_month)]
        
        df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 10, labels=False, duplicates='drop')
        
        df1_year_keymonth.info()
        
        df1_year_keymonth_size += len(df1_year_keymonth.index) 
        
        df1_group = df1_group.append(df1_year_keymonth, ignore_index=True)
        
        
df1_group

In [ ]:
df1_year_keymonth_size

In [ ]:
df1_group

df1_group.to_excel('加权权重分组.xlsx',index=False)



### Dataset df2_market_value

进行加权平均计算月度回报率（分组过后+加权之后）

匹配时间节点上的公司市值

匹配相应时间段的月度回报率数据


In [ ]:
# 进行加权平均计算月度回报率（分组过后+加权之后）
# 匹配时间节点上的公司市值
# 匹配相应时间段的月度回报率数据

df2_market_value = pd.DataFrame()
# loop years
for year in df2['统计日期'].dt.year.unique():
    print('*************')
    print(year)
    current_year_data = df2[df2['统计日期'].dt.year == year] # type: ignore
    print(current_year_data.head())
    
    # loop 季报 调仓时候的市值 
    for key, key_month in report_months.items():
        print('**********')
        print(key)
        print(key_month*4/3)
        current_year_df_monthly = current_year_data[current_year_data['统计日期'].dt.month == key_month*4/3]
        #print(current_year_df_monthly.info())
        #计算权重
    
        df2_market_value = df2_market_value.append(current_year_df_monthly, ignore_index=True)
        

In [ ]:
df2_market_value.reset_index()
df1_group.reset_index()


In [ ]:
df2_market_value

df2_market_value.to_excel('个股市值统计.xlsx',index=False)
df2_market_value

In [ ]:
df2_market_value

In [ ]:

# 假设你的DataFrame叫df，日期列叫'最新统计日期'
# 我们假设年份和日不变，只是月份改变了
df2_market_value['new_month'] = (df2_market_value['统计日期'].dt.month *3/4).astype(int)
df2_market_value['new_year'] = df2_market_value['统计日期'].dt.year
df2_market_value['new_day'] = 30  # 假设每月的第一天
df2_market_value
# 构建新的日期字符串
#df2_market_value['新统计日期'] = pd.to_datetime(df2_market_value[['new_year', 'new_month', 'new_day']])



In [ ]:


# 假设 new_year 和 new_month 已经定义
df2_market_value['最新统计日期'] = pd.to_datetime(df2_market_value['new_year'].astype(str) + '-' + df2_market_value['new_month'].astype(str) + '-01')
df2_market_value['最新统计日期'] = df2_market_value['最新统计日期'] + pd.offsets.MonthEnd()

In [ ]:
# df2_market_value['新统计日期'] = pd.to_datetime(df2_market_value[['new_year', 'new_month', 'new_day']])


# df2_market_value['最新统计日期'] = pd.to_datetime({
#     'year': df2_market_value['new_year'],
#     'month': df2_market_value['new_month'],
#     'day': df2_market_value['new_day']
# })

In [ ]:
df2_market_value.drop(columns=['new_year', 'new_month', 'new_day'], inplace=True)
df2_market_value


In [ ]:
df1_group['最新统计日期'] = df1_group['统计日期']

df1_group['证券代码'] = df1_group['证券代码'].astype(str)
df1_group['最新统计日期'] = pd.to_datetime(df1_group['最新统计日期'])

df2_market_value['证券代码'] = df2_market_value['证券代码'].astype(str)
df2_market_value['最新统计日期'] = pd.to_datetime(df2_market_value['最新统计日期'])

In [ ]:
df2_market_value.info()


In [ ]:
df1_group.head()

### df1_merged_group

merge df1_group, df2_market_value

In [ ]:

df1_merged_group = pd.merge(df1_group, df2_market_value, on=['证券代码','最新统计日期'],how='inner')
df1_merged_group


In [ ]:
# df1_merged_group_0 = df1_merged_group[df1_merged_group['分组'] == 0]
# df1_merged_group_0.head()


In [ ]:

# df1_merged_group_0_sum = df1_merged_group_0.groupby(by=["最新统计日期"]).sum()
# df1_merged_group_0_sum

### 个股 市值权重 in group and date

根据df1 group， 进行个股权重计算

In [ ]:
# # 取个股 市值 by date / 组内市值总和
# # save in a new dataframe
# # ['证券代码'] ['个股市值'] ['个股组内权重']
def getDF_stock_weight_group(df1_merged_group, df1_merged_group_sum):
    df_stock_weight_group = pd.DataFrame(columns=['证券代码', '个股市值', '个股组内权重','最新统计日期'] )
    
    for index, row in df1_merged_group.iterrows():
        # print(row['证券代码'])
        # print(row['个股市值'])
        # print(row['最新统计日期'])
        # print('group market value')
        group_row = df1_merged_group_sum[df1_merged_group_sum.index == row['最新统计日期']]
        # print(group_row['个股市值'])
        
        stock_weight = row['个股市值'] / group_row['个股市值'].item()   
        
        # print('stock_weight=======')
        # print(stock_weight)
        
        data = {'证券代码': row['证券代码'], '个股市值': row['个股市值'], '个股组内权重':stock_weight, '最新统计日期':row['最新统计日期'] }
        #print(data)
        #print(df_tmp.head())
    
        df_stock_weight_group = df_stock_weight_group.append(data, ignore_index=True)
    
    return df_stock_weight_group


In [ ]:

def getDF_stock_weight_group_by_number(index, df1_merged_group):
    
    df1_merged_group = df1_merged_group[df1_merged_group['分组'] == index]
    df1_merged_group_sum = df1_merged_group.groupby(by=["最新统计日期"]).sum()
    df_stock_weight_group = getDF_stock_weight_group(df1_merged_group, df1_merged_group_sum)   
    
    return df_stock_weight_group

In [ ]:
df_stock_weight_group0 = getDF_stock_weight_group_by_number(0, df1_merged_group)
df_stock_weight_group0

In [ ]:
df_stock_weight_group1 = getDF_stock_weight_group_by_number(1, df1_merged_group)
df_stock_weight_group1

In [ ]:
df_stock_weight_group2 = getDF_stock_weight_group_by_number(2, df1_merged_group)
df_stock_weight_group2

In [ ]:
df_stock_weight_group3 = getDF_stock_weight_group_by_number(3, df1_merged_group)
df_stock_weight_group3

In [ ]:
df_stock_weight_group4 = getDF_stock_weight_group_by_number(4, df1_merged_group)
df_stock_weight_group4

In [ ]:
#TODO 5
df_stock_weight_group5 = getDF_stock_weight_group_by_number(5, df1_merged_group)
df_stock_weight_group5

In [ ]:
# 6
df_stock_weight_group6 = getDF_stock_weight_group_by_number(6, df1_merged_group)
df_stock_weight_group6

In [ ]:
# 7 
df_stock_weight_group7 = getDF_stock_weight_group_by_number(7, df1_merged_group)
df_stock_weight_group7

In [ ]:
# 8
df_stock_weight_group8 = getDF_stock_weight_group_by_number(8, df1_merged_group)
df_stock_weight_group8

In [ ]:
# 9 
df_stock_weight_group9 = getDF_stock_weight_group_by_number(9, df1_merged_group)
df_stock_weight_group9

# Dataset 加权月度回报率

### Description
trd_month as datasource

每个 个股 交易时间

每个个股 在每个季度的权重 in df_stock_weight_groupN

个股组内权重 at timestamp （xxxx-03, xxxx-06, xxxx-09）

个股组内权重(4个月) * 个股月度回报率 （最新统计日期）

df_stock_weight_groupN

#### a new dataframe - seucrity code, datetime (df3_DATE), 统计日期-4 (alignment), weighted stock monthly return

#### weighted stock monthly return = weight (3 kinds) * monthly return (12 months)

monthly return (12 months) = get from df3 by df3[证券代码] and df3[统计日期-4]

weight (3 kinds) = df3[统计日期-4] --> 

{

xxxx-03 : (1-4)  2014-03 --> 

xxxx-06 : (5-8)  2014-06 --> 

xxxx-09 : (9-12) 2014-10 --> 

}




### fake code


combine df_stock_weight_group0 and add a new column for group

loop df3 

weighted stock monthly return = weight (3 kinds) * monthly return (12 months) --->
weight (3 kinds) --->  df_stock_weight_groupN ---> df_stock_weight_group0 + 1 + 2 + 3 +4

df_stock_weight_group0[最新统计日期] <-||->  df3[统计日期-4]
df_stock_weight_group0[证券代码] <--> df3[证券代码]

function (Date){


}


array [证券代码, 统计日期-4, DATE, 个股月回报率, weighted stock monthly return, 分组]


##### combine df_stock_weight_group0 and add a new column for group
df_stock_weight_groupN ---> df_stock_weight_group0 + 1 + 2 + 3 +4

In [ ]:
df_stock_weight_group0['group'] = 0
df_stock_weight_group1['group'] = 1
df_stock_weight_group2['group'] = 2
df_stock_weight_group3['group'] = 3
df_stock_weight_group4['group'] = 4
#TODO
df_stock_weight_group5['group'] = 5
df_stock_weight_group6['group'] = 6
df_stock_weight_group7['group'] = 7
df_stock_weight_group8['group'] = 8
df_stock_weight_group9['group'] = 9

In [ ]:
df_stock_weight_group0
df_stock_weight_group1

In [ ]:
df_stock_weight_group2
df_stock_weight_group3

In [ ]:
df_stock_weight_group4

In [ ]:
df_stock_weight_group5

In [ ]:
df_stock_weight_group6

In [ ]:
df_stock_weight_group = pd.concat([df_stock_weight_group9,df_stock_weight_group8,df_stock_weight_group7,df_stock_weight_group6,df_stock_weight_group5,df_stock_weight_group4, df_stock_weight_group3, df_stock_weight_group2, df_stock_weight_group1, df_stock_weight_group0], ignore_index=True)

df_stock_weight_group

##### subset df3
证券代码	DATE	统计日期-4	个股月回报率


In [ ]:
df3['DATE_str'] = df3['统计日期'].dt.date
df3['统计日期-4'] = df3['DATE_str'] - relativedelta(months = 4)
df3.head()

In [ ]:
df3_sub = df3[['证券代码','DATE_str', '统计日期-4','统计日期','个股月回报率']]
df3_sub

##### add a new column to map report date in df_stock_weight_group
{

xxxx-03 : (1-4) 2014-03 -->

xxxx-06 : (5-8) 2014-06 -->

xxxx-09 : (9-12) 2014-10 -->

}

In [ ]:

df3_sub['统计日期-4'] = df3_sub['统计日期-4'].astype('str')
df3_sub['统计日期-4'] = pd.to_datetime(df3_sub['统计日期-4'])

df3_sub.info()

##### update trading date
{

xxxx-03 : (1-4) 2014-03 -->

xxxx-06 : (5-8) 2014-06 -->

xxxx-09 : (9-12) 2014-10 -->

}

In [ ]:
def getTradingMonth(row):

    if(4 >= row['new_month'] >= 1):
        newMonth = 3
    elif(8>= row['new_month'] >=5 ):
        newMonth = 6
    else:
        newMonth = 9
    return newMonth


In [ ]:

df3_sub['new_month'] = (df3_sub['统计日期-4'].dt.month).astype(int)
df3_sub['new_trading_month'] = df3_sub.apply(getTradingMonth, axis=1)
df3_sub['new_year'] = df3_sub['统计日期-4'].dt.year
## end day
df3_sub['new_day'] = df3_sub['统计日期-4'].dt.day  



df3_sub['trading_date'] = pd.to_datetime({
    'year': df3_sub['new_year'],
    'month': df3_sub['new_trading_month'],
    'day': df3_sub['new_day']
})


In [ ]:
df3_sub_trading = df3_sub[['证券代码','DATE_str', '统计日期-4','统计日期','个股月回报率', 'trading_date']]
df3_sub_trading['最新统计日期'] = df3_sub_trading['trading_date'] + pd.offsets.MonthEnd()

df3_sub_trading

##### Append group 

##### merge df3_sub and df_stock_weight_group

column [证券代码, 统计日期-4, DATE, 个股月回报率, 分组]

['证券代码','DATE_str', '统计日期-4','统计日期','个股月回报率', 'trading_date']

In [ ]:
df_stock_weight_group.info()


In [ ]:
df3_sub_trading['证券代码'] = df3_sub_trading['证券代码'].astype(str)

df3_sub_trading['最新统计日期'] = pd.to_datetime(df3_sub_trading['最新统计日期'])
df_stock_weight_group['最新统计日期'] = pd.to_datetime(df_stock_weight_group['最新统计日期'])


In [ ]:
df3_sub_trading.head()


In [ ]:
df_stock_weight_group.head()

In [ ]:
df3_sub_trading_group = pd.merge(df3_sub_trading, df_stock_weight_group, on=['证券代码','最新统计日期'],how='left')

df3_sub_trading_group

In [ ]:
df3_sub_trading_group  = df3_sub_trading_group[['证券代码','统计日期-4','统计日期','trading_date','个股月回报率','个股组内权重','group']]

df3_sub_trading_group

##### weighted stock monthly return = weight (3 kinds) * monthly return (12 months)

array [证券代码, 统计日期-4, DATE, 个股月回报率, weighted stock monthly return, 分组]

In [ ]:
# df3_sub_trading_group.dropna()

df3_sub_trading_group['个股加权回报率'] = df3_sub_trading_group['个股月回报率']* df3_sub_trading_group['个股组内权重']
df3_sub_trading_group['组间加权回报率'] = df3_sub_trading_group['个股加权回报率']

df3_sub_trading_group_final = df3_sub_trading_group[df3_sub_trading_group['group'].notna()]

df3_sub_trading_group_final

In [ ]:
#df3_sub_trading_group_final.to_excel('df3_sub_trading_group_final.xlsx', index=False)


### 组间加权月度回报率
##### grouping weighted monthly return

according to the group, split the df

and group by DATE


In [ ]:
df3_sub_trading_group_final_0 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 0]
df3_sub_trading_group_final_0.info()
df3_sub_trading_group_final_0

In [ ]:
df3_sub_trading_group_final_0_group = df3_sub_trading_group_final_0.groupby(by=["统计日期"]).sum()
df3_sub_trading_group_final_0_group = df3_sub_trading_group_final_0_group[['组间加权回报率','group']]
df3_sub_trading_group_final_0_group

In [ ]:
df3_sub_trading_group_final_1 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 1]
df3_sub_trading_group_final_1.head()

In [ ]:
df3_sub_trading_group_final_1_group = df3_sub_trading_group_final_1.groupby(by=["统计日期"]).sum()

df3_sub_trading_group_final_1_group = df3_sub_trading_group_final_1_group[['组间加权回报率','group']]

df3_sub_trading_group_final_1_group['group'] = 1
df3_sub_trading_group_final_1_group

In [ ]:
df3_sub_trading_group_final_2 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 2]
df3_sub_trading_group_final_2.info()

In [ ]:
df3_sub_trading_group_final_2_group = df3_sub_trading_group_final_2.groupby(by=["统计日期"]).sum()

df3_sub_trading_group_final_2_group = df3_sub_trading_group_final_2_group[['组间加权回报率','group']]

df3_sub_trading_group_final_2_group['group'] = 2
df3_sub_trading_group_final_2_group

In [ ]:
df3_sub_trading_group_final_3 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 3]
df3_sub_trading_group_final_3.info()

In [ ]:
df3_sub_trading_group_final_3_group = df3_sub_trading_group_final_3.groupby(by=["统计日期"]).sum()

df3_sub_trading_group_final_3_group = df3_sub_trading_group_final_3_group[['组间加权回报率','group']]

df3_sub_trading_group_final_3_group['group'] = 3

df3_sub_trading_group_final_3_group

In [ ]:
df3_sub_trading_group_final_4 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 4]
df3_sub_trading_group_final_4.info()

In [ ]:
df3_sub_trading_group_final_4_group = df3_sub_trading_group_final_4.groupby(by=["统计日期"]).sum()

df3_sub_trading_group_final_4_group = df3_sub_trading_group_final_4_group[['组间加权回报率','group']]

df3_sub_trading_group_final_4_group['group'] = 4
df3_sub_trading_group_final_4_group


In [ ]:
#TODO 5
df3_sub_trading_group_final_5 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 5]
df3_sub_trading_group_final_5.info()

In [ ]:
df3_sub_trading_group_final_5_group = df3_sub_trading_group_final_5.groupby(by=["统计日期"]).sum()

df3_sub_trading_group_final_5_group = df3_sub_trading_group_final_5_group[['组间加权回报率','group']]

df3_sub_trading_group_final_5_group['group'] = 5
df3_sub_trading_group_final_5_group

In [ ]:
#TODO 6
df3_sub_trading_group_final_6 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 6]
df3_sub_trading_group_final_6.info()

In [ ]:
df3_sub_trading_group_final_6_group = df3_sub_trading_group_final_6.groupby(by=["统计日期"]).sum()

df3_sub_trading_group_final_6_group = df3_sub_trading_group_final_6_group[['组间加权回报率','group']]

df3_sub_trading_group_final_6_group['group'] = 6
df3_sub_trading_group_final_6_group

In [ ]:
#TODO 7
df3_sub_trading_group_final_7 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 7]
df3_sub_trading_group_final_7.info()

In [ ]:
df3_sub_trading_group_final_7_group = df3_sub_trading_group_final_7.groupby(by=["统计日期"]).sum()

df3_sub_trading_group_final_7_group = df3_sub_trading_group_final_7_group[['组间加权回报率','group']]

df3_sub_trading_group_final_7_group['group'] = 7
df3_sub_trading_group_final_7_group

In [ ]:
#TODO 8
df3_sub_trading_group_final_8 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 8]
df3_sub_trading_group_final_8.info()

In [ ]:
df3_sub_trading_group_final_8_group = df3_sub_trading_group_final_8.groupby(by=["统计日期"]).sum()

df3_sub_trading_group_final_8_group = df3_sub_trading_group_final_8_group[['组间加权回报率','group']]

df3_sub_trading_group_final_8_group['group'] = 8
df3_sub_trading_group_final_8_group

In [ ]:
#TODO 9
df3_sub_trading_group_final_9 = df3_sub_trading_group_final[df3_sub_trading_group_final['group'] == 9]
df3_sub_trading_group_final_9.info()

In [ ]:
df3_sub_trading_group_final_9_group = df3_sub_trading_group_final_9.groupby(by=["统计日期"]).sum()

df3_sub_trading_group_final_9_group = df3_sub_trading_group_final_9_group[['组间加权回报率','group']]

df3_sub_trading_group_final_9_group['group'] = 9
df3_sub_trading_group_final_9_group

In [ ]:
df3_sub_trading_group_final_0_group = df3_sub_trading_group_final_0_group.reset_index()
df3_sub_trading_group_final_1_group = df3_sub_trading_group_final_1_group.reset_index()
df3_sub_trading_group_final_2_group = df3_sub_trading_group_final_2_group.reset_index()
df3_sub_trading_group_final_3_group = df3_sub_trading_group_final_3_group.reset_index()
df3_sub_trading_group_final_4_group = df3_sub_trading_group_final_4_group.reset_index()
#TODO
df3_sub_trading_group_final_5_group = df3_sub_trading_group_final_5_group.reset_index()
df3_sub_trading_group_final_6_group = df3_sub_trading_group_final_6_group.reset_index()
df3_sub_trading_group_final_7_group = df3_sub_trading_group_final_7_group.reset_index()
df3_sub_trading_group_final_8_group = df3_sub_trading_group_final_8_group.reset_index()
df3_sub_trading_group_final_9_group = df3_sub_trading_group_final_9_group.reset_index()

In [ ]:
df3_sub_trading_group_final_0_group.info()

In [ ]:
# TODO
df3_sub_trading_group_final_group = pd.concat([df3_sub_trading_group_final_0_group,df3_sub_trading_group_final_1_group,df3_sub_trading_group_final_2_group,df3_sub_trading_group_final_3_group,df3_sub_trading_group_final_4_group,df3_sub_trading_group_final_5_group,df3_sub_trading_group_final_6_group,df3_sub_trading_group_final_7_group,df3_sub_trading_group_final_8_group,df3_sub_trading_group_final_9_group], ignore_index=True)

df3_sub_trading_group_final_group

In [ ]:
# df3_sub_trading_group_final_group04 = pd.concat([df3_sub_trading_group_final_0_group,df3_sub_trading_group_final_1_group,df3_sub_trading_group_final_2_group,df3_sub_trading_group_final_3_group,df3_sub_trading_group_final_4_group], ignore_index=True)
# df3_sub_trading_group_final_group04



In [ ]:

# df3_sub_trading_group_final_group59 = pd.concat([df3_sub_trading_group_final_5_group,df3_sub_trading_group_final_6_group,df3_sub_trading_group_final_7_group,df3_sub_trading_group_final_8_group,df3_sub_trading_group_final_9_group], ignore_index=True)
# df3_sub_trading_group_final_group59

In [ ]:
df3_sub_trading_group_final_group.info()

In [ ]:
df3_sub_trading_group_final_group['统计日期'] = df3_sub_trading_group_final_group['统计日期'].apply(lambda x: x.strftime('%Y-%m-%d'))
df3_sub_trading_group_final_group

In [ ]:
df3_sub_trading_group_final_group.to_excel('group_weighted_monthly_return.xlsx', index=False)


# Comment